Spatial ICA on source‐reconstructed EEG

In [4]:
import numpy as np
import sklearn
import mne

In [ ]:
# Source-reconstruct your cleaned EEG
# assuming you’ve already built an inverse operator in MNE…
stc = inverse_operator.apply_inverse_raw(raw, lambda2=1/9, method='dSPM')

In [ ]:
# Restrict to your fMRI‐active vertices (vertices of interest)
label = mne.read_label('fmri_active.label')         # a label made from your fMRI mask  
verts = [label.vertices]                            # list-of-lists per hemisphere  
data_roi = np.vstack([stc.data[v, :] for v in verts])  # shape = (n_voxels_in_mask, n_times)

In [ ]:
# Reduce dimensionality with PCA
from sklearn.decomposition import PCA
X = data_roi.T                # shape = (n_times, n_voxels)
pca = PCA(n_components=20)    
Xr = pca.fit_transform(X)     # now (n_times, 20)

In [ ]:
# Run ICA on the PCA‐reduced data
from sklearn.decomposition import FastICA
ica = FastICA(n_components=10)
S = ica.fit_transform(Xr)     # (n_times, 10)    ← time-courses of ICs
A = ica.mixing_               # (20, 10)         ← mixing from PCA space to ICs

In [ ]:
# Back-project each IC into cortex space
# ica.components_ is (10, 20); invert PCA to get (10, n_voxels)
spatial_maps = pca.inverse_transform(ica.components_)  

In [ ]:
# Select the IC(s) whose spatial map best correlates with your fMRI map
# assume fmri_map is a vector of t-values at each voxel
corrs = [ np.corrcoef(spat, fmri_map)[0,1] 
          for spat in spatial_maps ]
best = np.argmax(np.abs(corrs))   # pick IC with highest |corr|
roi_timecourse = S[:, best]

In [ ]:
# Threshold or cluster that IC’s spatial map if you still want a binary segmentation.

Outputs:
EEG Data
Valence
Arousal
Audio